![Qiskit](https://github.com/Qiskit/qiskit-tutorials/raw/115c78962dda85bac29d679063b7d0d0ab1d1ab4/images/qiskit-heading.gif)

# UniB Workshop 26/27 - Qiskit Aqua
## IBMQ
### François Varchon (francois.varchon@fr.ibm.com)
##### Thanks to Donny Greenberg




# Gameplan

* Overview and basics
  * What is Aqua?
  * Structural Elements
  * Control flow
  * Interfaces
* Detailed component overview
* Tips and Tricks
* Learning more

But first, install Aqua:

In [ ]:
!pip install qiskit-aqua

# What is Aqua?

Aqua is:
* An easy to use library for running many different quantum algorithms
* A collection of reusable components which can be mixed and matched in different algorithms

Aqua is very practically driven
* It is exactly what you’d expect to spring out of rapid practical need… it is very function-driven and evolves rapidly
* The guts are not so inviting in some places because of how rapidly it evolves and because it is engineered and optimized to the nines
* If you need to do something, there is a good chance someone already put it in Aqua or should
  * Please send feature requests!

Aqua is built to be highly extensible and have very easy to use interfaces (more on this next)
Four primary interfaces
* Creating a QuantumAlgorithm object in python with a config dict (most popular) and calling run()
* Aqua GUI
* Creating QuantumAlgorithm and setting params manually
* Command line interface


# Plug and play components, not a circuit library

Keep in mind:

Aqua organizes reusable components into buckets, elements of which can be substituted for one another:
* Algorithms
* Variational forms
* Optimizers
* Other smaller buckets (translators, QFTs, oracles, feature maps)
It is not so much a “I give you parameters, you give me back a circuit” library, because in most cases, running the algorithms is much more than just executing the circuit (e.g. VQE, q-kernel SVM, etc.)

It’s not even so much about “give me x component,” it’s more “execute this algorithm, with this set of components,” allowing you to mix and match or add your own components easily

There are many tests, which give good examples of execution modes and components, as well as tutorials in the qiskit-tutorials repo

Get an IDE (Pycharm is good!) and step through the code!


# Structural Elements

Aqua is organized in roughly four high-level elements:
* Algorithms - the control flow and logic in generally well contained inside the algorithm class file. 
* Interfaces - Aqua is meant to be highly accessible, so it has a lot of infrascructure to allow the graphical or JSON interfaces to be almost equally accessible as coding against the library directly 
   * JSON definitions and declarative interfaces are dispersed throughout
   * The UI and CLI are in the top level Aqua directory
* Components - These are objects which are shared by several algorithms.
* Utilities - Aqua has many useful modules and helper classes that an algorithm and application developer might want. Examples include the [operator class](https://github.com/Qiskit/qiskit-aqua/tree/master/qiskit/aqua/operator.py), [random matrix generator](https://github.com/Qiskit/qiskit-aqua/tree/master/qiskit/aqua/utils/random_matrix_generator.py), [run_circuits.py](https://github.com/Qiskit/qiskit-aqua/tree/master/qiskit/aqua/utils/run_circuits.py), and much more.

# Exemple 1: The Maxcut problem

Let's focus on a combinatorial optimization problem called **Maxcut**, which is solved by dividing the nodes of a weighted graph into two groups such that the egdes between the two groups carry the most possible weight. There's an [excellent notebook](https://github.com/Qiskit/qiskit-tutorial/blob/master/qiskit/aqua/optimization/maxcut_and_tsp.ipynb) by Mezzacapo et al that goes into the problem in more detail, which I highly recommend. For our purposes, reviewing solution paths is a more important use of our time, and we will not be spending much time on the translation between the graph problem and the qubit operator.

Our solution path starts like this - we represent each node on the graph by a qubit, and the 0 or 1 state of the qubit represents which of the two groups each node occupies in a possible solution. Binary strings are possible solutions, describing how to divide the nodes into |0> group and |1> group. We can take the weights of the graph and turn them into a cost matrix on the vector space of these strings, such that this cost matrix represents an Ising hamiltonian whose minimum eigenstate is our optimal solution.

Let's begin with a weight matrix w, representing the weights between edges of the graph. To make it easier to follow along with the above mentioned notebook, we'll use the same cost matrix.

In [ ]:
# Housekeeping: uncomment this to suppress deprecation warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
from qiskit.tools.visualization import plot_histogram
from qiskit_aqua import Operator, run_algorithm
from qiskit_aqua.input import EnergyInput
from qiskit_aqua.translators.ising import maxcut
import networkx as nx

from qiskit import Aer
from qiskit.tools.visualization import plot_histogram
from qiskit_aqua import Operator, run_algorithm
from qiskit_aqua.input import EnergyInput
from qiskit_aqua.translators.ising import maxcut
from qiskit_aqua.algorithms import VQE, ExactEigensolver
from qiskit_aqua.components.optimizers import SPSA, SLSQP
from qiskit_aqua.components.variational_forms import RY,RYRZ
from qiskit_aqua import QuantumInstance


# setup aqua logging
import logging
from qiskit_aqua._logging import set_logging_config, build_logging_config

In [ ]:
w = np.array([[0., 1., 1., 1.,],
 [1., 0., 1., 0.,],
 [1., 1., 0., 1.,],
 [1., 0., 1., 0.,]])

qubitOp, offset = maxcut.get_maxcut_qubitops(w)
algo_input = EnergyInput(qubitOp)


In [ ]:
n=4 # Number of nodes in graph
G=nx.Graph()
elist=[(0,1,1.0),(0,2,1.0),(0,3,1.0),(1,2,1.0),(2,3,1.0)]
# tuple is (i,j,weight) where (i,j) is the edge
G.add_weighted_edges_from(elist)
pos = nx.spring_layout(G)

For simplicity, we'll be using the declarative interface to Aqua today, which is accessed via a configuration dictionary.

In [ ]:
# Dictionnary approach
aqua_dict = {
    'problem': {'name': 'ising', 'random_seed': 10598},
    'algorithm': {'name': 'VQE','operator_mode': 'matrix'},
    'optimizer': {'name': 'SPSA','max_trials': 300},
    'variational_form': {'name': 'RY','depth': 5,'entanglement': 'linear'},
    'backend': {'name': 'statevector_simulator'}
}

result = run_algorithm(aqua_dict, algo_input)

''' # Programmatic approach
seed = 10598
spsa = SPSA(max_trials=300)
ry = RY(qubitOp.num_qubits, depth=5, entanglement='linear')
vqe = VQE(qubitOp, ry, spsa, 'matrix')

backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend=backend, seed=seed, seed_mapper=seed)

result = vqe.run(quantum_instance)
'''


x = maxcut.sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('maxcut objective:', result['energy'] + offset)
print('solution:', maxcut.get_graph_solution(x))
print('solution objective:', maxcut.maxcut_value(x, w))

sol = maxcut.get_graph_solution(x)
sol

In [ ]:
colors = ['r' if maxcut.get_graph_solution(x)[i] == 0 else 'b' for i in range(n)]
nx.draw_networkx(G, node_color=colors, node_size=600, alpha = .8, pos=pos)

Now, let's try a new optimizer. This is a big part of the power of Aqua. The Aqua team spends time making the components broadly compatable, so experimentation is cheap and easy.

In [ ]:
# Declarative approach
aqua_dict = {
    'problem': {'name': 'ising', 'random_seed': 10598},
    'algorithm': {'name': 'VQE','operator_mode': 'matrix'},
    'optimizer': {'name': 'SLSQP'},
    'variational_form': {'name': 'RY','depth': 5,'entanglement': 'linear'},
    'backend': {'name': 'statevector_simulator'}
}

result = run_algorithm(aqua_dict, algo_input)
'''
seed = 10598
slsqp=SLSQP()
ry = RY(qubitOp.num_qubits, depth=5, entanglement='linear')
vqe = VQE(qubitOp, ry, slsqp, 'matrix')

backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend=backend, seed=seed, seed_mapper=seed)

result = vqe.run(quantum_instance)
'''
x = maxcut.sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('maxcut objective:', result['energy'] + offset)
print('solution:', maxcut.get_graph_solution(x))
print('solution objective:', maxcut.maxcut_value(x, w))

sol = maxcut.get_graph_solution(x)
sol

In [ ]:
colors = ['r' if maxcut.get_graph_solution(x)[i] == 0 else 'b' for i in range(n)]
nx.draw_networkx(G, node_color=colors, node_size=600, alpha = .8, pos=pos)

Empirical notes on optimizers:
* COBYLA is a generally good global optimizer, especially in situations when you do not have a good starting point
* SLSQP is very good when you have a good starting point, and the error surface can be approximated in small cavities by quadratic functions
* SPSA is generally resilient to noise, so almost always the first choise for running on the hardware or noisy simulations

Now let's try a new variational form:

In [ ]:
# Declarative approach
aqua_dict = {
    'problem': {'name': 'ising', 'random_seed': 10598},
    'algorithm': {'name': 'VQE','operator_mode': 'matrix'},
    'optimizer': {'name': 'SLSQP'},
    'variational_form': {'name': 'RYRZ','depth': 5,'entanglement': 'linear'},
    'backend': {'name': 'statevector_simulator'}
}
result = run_algorithm(aqua_dict, algo_input)

''' # Programmatic approach
seed = 10598
slsqp=SLSQP()
ryrz = RYRZ(qubitOp.num_qubits, depth=5, entanglement='linear')
vqe = VQE(qubitOp, ryrz, slsqp, 'matrix')

result = vqe.run(quantum_instance)
'''

x = maxcut.sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('maxcut objective:', result['energy'] + offset)
print('solution:', maxcut.get_graph_solution(x))
print('solution objective:', maxcut.maxcut_value(x, w))

sol = maxcut.get_graph_solution(x)
sol

That seemed to work worse. We can also increase variational form depth:

In [ ]:
aqua_dict = {
    'problem': {'name': 'ising', 'random_seed': 10598},
    'algorithm': {'name': 'VQE','operator_mode': 'matrix'},
    'optimizer': {'name': 'SLSQP'},
    'variational_form': {'name': 'RY','depth': 8,'entanglement': 'linear'},
    'backend': {'name': 'statevector_simulator'}
}

result = run_algorithm(aqua_dict, algo_input)

x = maxcut.sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('maxcut objective:', result['energy'] + offset)
print('solution:', maxcut.get_graph_solution(x))
print('solution objective:', maxcut.maxcut_value(x, w))

sol = maxcut.get_graph_solution(x)
sol

In many cases we can even swap out the **algorithm**:

In [ ]:
aqua_dict = {
    'problem': {'name': 'ising', 'random_seed': 10598},
    'algorithm': {'name': 'QAOA.Variational','operator_mode': 'matrix'},
    'optimizer': {'name': 'COBYLA'},
    'backend': {'name': 'statevector_simulator'}
}

result = run_algorithm(aqua_dict, algo_input)

x = maxcut.sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('maxcut objective:', result['energy'] + offset)
print('solution:', maxcut.get_graph_solution(x))
print('solution objective:', maxcut.maxcut_value(x, w))

sol = maxcut.get_graph_solution(x)
sol

# Interfaces: Declaration vs. Programmatic

The form above relies on a dictionary to construct the Aqua components, and is convenient for many use cases. The dictionary allows you to experiment with low-risk changes to the algorithm in a cheap and straightforward way. 

In addition, objects can be constructed explicitly via imports and constructors. This allows you to reuse or modify components more daringly, and most algorithms developers eventully take this route for many parts of their work.

# Exemple 2: Using Grover Search for satisfiability problems¶

This notebook demonstrates how to use the `Qiskit Aqua` library Grover algorithm and process the result.

Grover search is one of the most popular algorithms used for searching a solution among many possible candidates using Quantum Computers. If there are $N$ possible solutions among which there is exactly one solution (that can be verified by some function evaluation), then Grover search can be used to find the solution with $O(\sqrt{N})$ function evaluations. This is in contrast to classical computers that require $\Omega(N)$ function evaluations: the Grover search is a quantum algorithm that provably can be used search the correct solutions quadratically faster than its classical counterparts.  

Here, we are going to illustrate the use of Grover search to solve a combinatorial problem called 3 SAT problem (https://en.wikipedia.org/wiki/Boolean_satisfiability_problem ). The 3-SAT problem is a NP-complete problem, namely, it is one of the most difficult problems that are interconnected (meaning that if we solve any one of them, we essentially can solve all of them). Unfortunately, there are many natural problems that are NP-complete, such as, the Traveling Salesman Problem (TSP), the Maximum Cut (MaxCut) and so on. Up to now, there is no classical and quantum algorithm that can efficiently solve such NP-hard problems. 


##  3-SAT problem

The 3-SAT problem is best explained with the following concrete problem. Let us consider a Boolean function $f$ with three Boolean variables $x_1, x_2, x_3$ as below.

$$
f(x_1, x_2, x_3) = (\neg x1 \vee \neg x2 \vee \neg x3) \wedge (x1 \vee \neg x2 \vee x3)\wedge(x1 \vee x2 \vee \neg x3)\wedge(x1 \vee \neg x2 \vee \neg x3)\wedge(\neg x1 \vee x2 \vee x3)
$$

In the above function, the terms on the right-hand side equation which are inside $()$ are called clauses. Each clause has exactly three literals. Namely, the second clause has $x_1$, $\neg x_2$ and $ x_3$ as its literals. The symbol $\neg$ is the Boolean NOT that negates (or, flips) the value of its succeeding literal. The symbols $\vee$ and $\wedge$ are, respectively, the Boolean OR and AND. The Boolean $f$ is satisfiable if there is an assignment of $x_1, x_2, x_3$ that evaluates to $f(x_1, x_2, x_3) = 1$ (or, $f$ evaluates to True). 

In [ ]:
import pylab
import numpy as np
from qiskit import LegacySimulators
from qiskit.tools.visualization import plot_histogram
from qiskit_aqua import QuantumInstance
from qiskit_aqua import run_algorithm
from qiskit_aqua.algorithms import Grover
from qiskit_aqua.components.oracles import SAT

We have a SAT problem to which we want to find solutions using Grover and SAT oracle combination. The SAT problem is specified in the DIMACS CNF format. We read one of the sample cnf files to load the problem.

In [ ]:
with open('3sat3-5.cnf', 'r') as f:
    sat_cnf = f.read()
print(sat_cnf)

With this problem input, we create the corresponding `oracle` component:

In [ ]:
sat_oracle = SAT(sat_cnf)

The `oracle` can now be used to create an Grover instance:

In [ ]:
grover = Grover(sat_oracle)

We can then configure the backend and run the Grover instance to get the result:

In [ ]:
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=100)
result = grover.run(quantum_instance)
print(result['result'])

As seen above, a satisfying solution to the specified sample SAT problem is obtained, with the absolute values indicating the variable indices, and the signs the `True/False` assignments, similar to the DIMACS format.

A measurements result is also returned where it can be seen, below in the plot, that `result['result']` was the highest probability. But the other solutions were very close in probability too.

In [ ]:
plot_histogram(result['measurements'])

The above step-by-step programatic approach can also be achieved by using a json configuration dictionary with the parameters for the algorithm and any other dependent objects it requires, as follows:

In [ ]:
params = {
    'problem': {'name': 'search'},
    'algorithm': {
        'name': 'Grover'
    },
    'oracle': {
        'name': 'SAT',
        'cnf': sat_cnf
    },
    'backend': {
        'shots': 100
    }
}

result_dict = run_algorithm(params, backend=backend)
plot_histogram(result_dict['measurements'])

# Other Neat Stuff

Aqua has many core algorithms, including:
* VQE
* QAOA
* QPE/iterativeQPE
* Hamiltonian Evolution
* Quantum-Kernel SVM
* Variational SVM
* Grover’s
* Qiskit aqua ui

# Tips and tricks (Basically the same as Terra...)

* Put many circuits into a single execution!
  * Simulators will (generally) execute these in parallel
  * Quantum Hardware does a lot of calibration for each new job, so if you send 100 jobs it will generally take 100x as long as one job with 100 circuits, even if the circuits are completely different!
* The “qasm_simulator” (cpp) backend fails gracefully over to the “qasm_simulator_py,” which is ~5x slower!
  * (ctrl-f for 'failing gracefully')
* Consider commenting out Qobj validation if you need more speed in an iterative algo, but don’t tell anyone who told you so!
* Use an IDE!! A lot of people at IBM use Pycharm. Being able to step through the code is critical!
* Look at the debug log messages. There is a ton of important info in there. See notebook here
  * Even better, save them to a file.

In [ ]:
import logging
logging.getLogger('qiskit').setLevel(logging.DEBUG)

Redirecting logs to a file:

```
# Redirecting debug logs to a file (can't be done in colab):
    loggerc = logging.getLogger('qiskit_aqua_chemistry')
    loggerc.setLevel(logging.DEBUG)
    loggera = logging.getLogger('qiskit_aqua')
    loggera.setLevel(logging.DEBUG)
    loggerq = logging.getLogger('qiskit')
    loggerq.setLevel(logging.DEBUG)
    formatter = logging.Formatter(fmt='%(asctime)s %(levelname)-8s %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    hdlr = logging.FileHandler(outdir + log_file_name, mode='w')
    hdlr.setFormatter(formatter)
    loggerc.addHandler(hdlr)
    loggera.addHandler(hdlr)
    loggerq.addHandler(hdlr)
    print('\nlog file: {}'.format(outdir + log_file_name))
# <build, execute, etc.>
# close up handlers
    loggerc.removeHandler(hdlr)
    loggera.removeHandler(hdlr)
    loggerq.removeHandler(hdlr)
    hdlr.close()
```

# Part II: Aqua Chemistry

# Aqua Chemistry is a Convenient Wrapper Around Aqua and Popular Chemistry Drivers

* Aqua Chemistry is an even more practitioner-centric tool, specifically for Chemists
* It includes two other component buckets that are not in Aqua: Chemistry drivers, and mappings (Jordan-Wigner, Bravyi-Kitaev, parity, etc.)
* The interfaces look very similar to Aqua - instantiation with a dictionary, or a standalone UI, etc.
* The core service of Aqua Chemistry is to prepare all of the required calculations for the hamiltonian and other important observables before attempting to do Chemistry simulation or eigendecomposition in Aqua


In [ ]:
!pip install qiskit-chemistry
#!pip install pyscf

### Using aqua chemistry to compute H$_2$

This notebook demonstrates how to use Qiskit Chemistry to compute the ground state energy of molecular Hydrogen (H$_2$) using the Variational Quantum Eigensolver (VQE) algorithm and the Unitary Coupled Cluster Singles and Doubles (UCCSD) variational form.  This notebook uses the so called *declarative approach*: a Python dictionary  summarizes the entire experiment declaratively (could be automatically generated via the Qiskit Chemistry GUI wizard).  That dictionary is simply then passed as a paramter to the `run` method of the `AquaChemistry` solver to get the result of the experiment, also in the form of a Python dictionary.


This notebook has been written to use the HDF5 chemistry driver. This driver uses molecular data that has been serialized from a prior computation.  This allows this notebook to be executed with no additional driver installation requirements. See the Qiskit Chemistry driver documentation for more detail.

First, we import `QiskitChemistry`, which is the object that will carry out the computation for us

In [ ]:
import numpy as np
import pylab

from qiskit_chemistry import QiskitChemistry
from qiskit import Aer

Next, we create a Python dictionary to specify the problem we want to solve. There are defaults for many additional values that are not shown here for simpicity. Indeed, we take advantage of using sensisble defaults that the Qiskit Chemistry stack provides.  As mentioned above, the Qiskit Chemistry GUI wizard allows for automatic extraction of the Python dictionary, reflecting the current configuration.  Once the Python dictionary has been extracted, it can be pasted into a Python program or a Jupyter Notebook and, if necessary, edited.

The first entry names a chemistry driver. **Currently, Qiskit Chemistry comes with interfaces prebuilt for the following four computational chemistry software drivers (Gaussian 16, PSI4, PySCF, PyQuante)**. Here we use HDF5 and the next line configures the driver for an `hdf5` file that contains data from a prior computation for an H$_2$ molecule with basis set `sto-3g`. The operator line would be automatically set by default, but we have added it here to show it and to emphasize where the problem input is converted into a quantum qubit form. We then indicate to the `QiskitChemistry` solver that the VQE algorithm should be used in this experiment, using the COBYLA optimizer with a UCCSD variatonal form and initial state of HartreeFock. VQE uses the variational method to find the mimimum eigenvalue of a problem, which in this case is the ground state energy of the molecule.

In [ ]:
from qiskit import Aer
backend = Aer.get_backend('statevector_simulator')

In [ ]:
# Input dictionary to configure Qiskit AQUA Chemistry for the chemistry problem.

qiskit_chemistry_dict = {
    'driver': {'name': 'HDF5'},
    'HDF5': {'hdf5_input': 'H2/0.7_sto-3g.hdf5'},
    'operator': {'name': 'hamiltonian'},
    'algorithm': {'name': 'VQE'},
    'optimizer': {'name': 'COBYLA'},
    'variational_form': {'name': 'UCCSD'},
    'initial_state': {'name': 'HartreeFock'}
}

We can now create an `QiskitChemistry` object and call `run` on it passing in the problem dictionary to get a result. This may take a short time and it will use a local quantum simulator to carry out the quantum computation that the VQE algorithm uses.

In [ ]:
solver = QiskitChemistry()
result = solver.run(qiskit_chemistry_dict, backend=backend)

The `run` method returns a result dictionary. Some notable fields include `energy`, which is the computed ground state energy. We can print it.

In [ ]:
print('Ground state energy: {}'.format(result['energy']))

There is also a `printable` field containing a complete ready to print readable result.

In [ ]:
for line in result['printable']:
    print(line)

This was a very simple example showing how to get started. There are more elaborate notebooks here as well documentation describing the various components and their configurations to help you to experiment with quantum computing and its application to solving chemistry problems.

To continue : 
https://github.com/Qiskit/qiskit-tutorials/tree/master/qiskit/aqua/chemistry (Mezzacapo et al notebooks)

---------------------------------------------------------------------------------------------------------------------------

# Most Important Thing to Grasp - the Execution Flow:

* Aqua Chemistry discovers components and reads config dictionaries
* Most fancy Chemistry package features are allowed, like orbital removal, qubit tapering, core freezing, etc.
* Calls Chemistry Drivers to calculate single and double excitation integrals
* Uses mapping to create a hamiltonian circuit out of operators for each excitation
* Calls VQE in Aqua to find eigenstates of the hamiltonian (e.g. ground state energy)
  * Also passes aux_ops (auxiliary operators) for Aqua to calculate at the end: spin, particle number, dipole moment, etc.
* Aqua creates an “evaluate_energy” function which: 
  * Takes in a scalar parameter list, creates a variational form (UCCSD, RyRz, Swap-Rz, your own, etc.) parameterized by the list
  * Executes the variational circuit + the hamiltonian circuit (or does math in the background to simulate the result quickly without sending the full circuit to the simulator)
  * Returns the resulting energy of the system
* Aqua instantiates an optimizer, and passes the energy function as the cost function to minimize
  * Use COBYLA if running on statevector simulator, SPSA if running on shot-based (qasm) because it performs well in the presence of noise
* The optimizer minimizes the energy, and returns the optimal parameters when complete
* Aqua re-creates the ground state with the optimal params, and calculates and returns the ground state energy and auxiliary operators

# Tips!

* You don’t need to use VQE to find eigenstates! You can use QPE, QAA, or whatever else
* Your initial point matters a lot. The way you move along a dissociation curve can dictate the fate of your resutlts.
* UCCSD is nice but it will not be runnable on Quantum hardware for a long time.
* Optimization over non-convex high-dimensional error surfaces is hard. I am sorry about that.